## ***Pandas-to-SQL EV Pipeline with Analytics***

In [1]:
import pandas as pd
import sqlite3

In [2]:
# Load CSV into a single DataFrame
df = pd.read_csv('../data/final_vehicle_data.csv')

In [3]:
# Table definitions: name -> list of columns to extract
TABLE_COLS = {
"BasicInfo":      ["brand", "car_model", "segment", "drivetrain", "seats"],
"BatteryInfo":    ["car_model", "battery_capacity_kWh", "number_of_cells", "battery_type"],
"PerformanceInfo":["car_model", "top_speed_kmh", "acceleration_0_100_s", "torque_nm",
                   "torque_nm_mapped", "power_to_weight_ratio", "power_to_weight_category"],
"EfficiencyInfo": ["car_model", "efficiency_wh_per_km", "efficiency", "fast_charging_power_kw_dc", "fast_charge_port"],
"RangeInfo":      ["car_model", "range_km", "towing_capacity_kg", "cargo_volume_l"],
"DimensionsInfo": ["car_model", "length_mm", "width_mm", "height_mm", "car_body_type", "manufacturing_country"],
}

# Create SQLite DB and materialize normalized tables from the single DataFrame
with sqlite3.connect("cars.db") as conn:
    for tbl, cols in TABLE_COLS.items():
        df.loc[:, cols].drop_duplicates().to_sql(tbl, conn, if_exists="replace", index=False)
        
# Index join key for performance
conn.execute("CREATE INDEX IF NOT EXISTS idx_basic_car_model ON BasicInfo(car_model);")
conn.execute("CREATE INDEX IF NOT EXISTS idx_perf_car_model ON PerformanceInfo(car_model);")

In [4]:
# Define a multi-line SQL query in Python 
query = """
-- Create a CTE (Common Table Expression) named Bi that selects all columns
-- from the BasicInfo table. A CTE can make queries easier to read and reuse.
WITH Bi AS (SELECT * FROM BasicInfo)
SELECT
    Bi.brand,
    Bi.car_model,
    Bi.segment,
    Ba.battery_capacity_kWh,
    P.top_speed_kmh,
    E.efficiency,
    R.range_km,
    D.car_body_type
FROM Bi
LEFT JOIN BatteryInfo    AS Ba USING(car_model)
LEFT JOIN PerformanceInfo AS P USING(car_model)
LEFT JOIN EfficiencyInfo AS E USING(car_model)
LEFT JOIN RangeInfo      AS R USING(car_model)
LEFT JOIN DimensionsInfo AS D USING(car_model)
;
"""
# # Execute the SQL and load result into a pandas DataFrame.
combined = pd.read_sql(query, conn)

# Aggregation: compute the overall average top speed across all rows (per market segment)
agg_overall = pd.read_sql("SELECT AVG(top_speed_kmh) AS avg_top_speed FROM PerformanceInfo", conn)
agg_by_segment = pd.read_sql("""
SELECT Bi.segment, AVG(P.top_speed_kmh) AS avg_top_speed
FROM BasicInfo AS Bi
JOIN PerformanceInfo AS P USING(car_model)
GROUP BY Bi.segment
ORDER BY avg_top_speed DESC
""", conn)

print("Combined sample (first 10 rows):")
print(combined.head(10))
print("\nOverall average top speed:")
print(agg_overall)
print("\nAverage top speed by segment:")
print(agg_by_segment)

Combined sample (first 10 rows):
     brand                car_model            segment  battery_capacity_kWh  \
0    Dacia       Spring Electric 45           A - Mini                  25.0   
1    Dacia       Spring Electric 65           A - Mini                  25.0   
2  Citroen  e-SpaceTourer XL 50 kWh  N - Passenger Van                  46.3   
3  Peugeot    e-Traveller L2 75 kWh  N - Passenger Van                  68.0   
4  Peugeot    e-Traveller L3 50 kWh  N - Passenger Van                  46.3   
5  Peugeot    e-Traveller L3 75 kWh  N - Passenger Van                  68.0   
6  Citroen  e-SpaceTourer XL 75 kWh  N - Passenger Van                  68.0   
7  Citroen   e-SpaceTourer M 75 kWh  N - Passenger Van                  68.0   
8  Renault      5 E-Tech 40kWh 95hp        B - Compact                  40.0   
9  Citroen   e-SpaceTourer M 50 kWh  N - Passenger Van                  46.3   

   top_speed_kmh  efficiency  range_km        car_body_type  
0            125    2.88

In [5]:
# Window functions: This query computes per-segment percentile values for top speed using a
# "nearest-rank" fallback method (works in plain SQL without percentile functions).
percentiles_fallback_query = """
-- Build a merged set with ordering and counts per segment.
WITH merged AS (
  SELECT Bi.segment, P.car_model, P.top_speed_kmh, E.efficiency,
         ROW_NUMBER() OVER (PARTITION BY Bi.segment ORDER BY P.top_speed_kmh) AS rn_ts,
         COUNT(*) OVER (PARTITION BY Bi.segment) AS cnt
  FROM BasicInfo AS Bi
  JOIN PerformanceInfo AS P USING(car_model)
  JOIN EfficiencyInfo  AS E USING(car_model)
),

-- Precompute nearest-rank index positions for desired percentiles.
ranked AS (
  SELECT
    segment, top_speed_kmh, cnt,
    -- compute 1-based indices for 10th, 50th, 90th percentiles (nearest-rank)
    CAST( (cnt - 1) * 0.10 + 1 AS INTEGER ) AS idx10,
    CAST( (cnt - 1) * 0.50 + 1 AS INTEGER ) AS idx50,
    CAST( (cnt - 1) * 0.90 + 1 AS INTEGER ) AS idx90,
    rn_ts
  FROM merged
)

--Extract percentile values by matching row numbers to computed indices.
SELECT
  r.segment,
  MAX(CASE WHEN rn_ts = idx10 THEN top_speed_kmh END) AS p10_top_speed,
  MAX(CASE WHEN rn_ts = idx50 THEN top_speed_kmh END) AS p50_top_speed,
  MAX(CASE WHEN rn_ts = idx90 THEN top_speed_kmh END) AS p90_top_speed,
  -- median efficiency per segment (nearest-rank using same logic)
  (SELECT top_speed_kmh FROM merged m2 WHERE m2.segment = r.segment AND m2.rn_ts = CAST((r.cnt-1)*0.50+1 AS INTEGER) LIMIT 1) AS median_efficiency_placeholder
FROM ranked r
GROUP BY r.segment, r.cnt
ORDER BY p50_top_speed DESC;
"""
percentiles_by_segment = pd.read_sql(percentiles_fallback_query, conn)
print("\nPercentiles by segment:")
print(percentiles_by_segment)


Percentiles by segment:
              segment  p10_top_speed  p50_top_speed  p90_top_speed  \
0          F - Luxury            200            250            270   
1          I - Luxury            250            250            250   
2      JE - Executive            200            210            240   
3       E - Executive            180            200            215   
4          JD - Large            170            200            220   
5         JF - Luxury            180            200            250   
6          G - Sports            195            195            195   
7           D - Large            170            190            210   
8         JC - Medium            160            180            185   
9        JB - Compact            150            175            190   
10         C - Medium            150            160            185   
11        B - Compact            135            150            160   
12          JA - Mini            140            140            14

In [6]:
# Build a composite score ranking (top 10) from several normalized metrics
composite_query = """
-- Collect the raw metrics we need per model.
WITH metrics AS (
  SELECT
    Bi.car_model,
    Bi.brand,
    P.power_to_weight_ratio,
    E.efficiency_wh_per_km,
    Ba.battery_capacity_kWh
  FROM BasicInfo AS Bi
  JOIN PerformanceInfo AS P USING(car_model)
  JOIN EfficiencyInfo  AS E USING(car_model)
  JOIN BatteryInfo     AS Ba USING(car_model)
),

-- Compute min/max for each metric to use in min-max normalization
bounds AS (
  SELECT
    MIN(power_to_weight_ratio) AS min_pw, MAX(power_to_weight_ratio) AS max_pw,
    MIN(efficiency_wh_per_km) AS min_eff, MAX(efficiency_wh_per_km) AS max_eff,
    MIN(battery_capacity_kWh) AS min_bat, MAX(battery_capacity_kWh) AS max_bat
  FROM metrics
),

-- normalize each metric to a 0..1 range using min-max scaling
-- For efficiency (lower is better) we invert the scale so higher normalized = better
-- If a metric has zero range (max = min) we assign a neutral value 0.5 to avoid division by zero
norms AS (
  SELECT
    m.car_model, m.brand, m.power_to_weight_ratio, m.efficiency_wh_per_km, m.battery_capacity_kWh,
    -- min-max normalize; if range=0 set neutral 0.5
    CASE WHEN b.max_pw = b.min_pw THEN 0.5 ELSE (m.power_to_weight_ratio - b.min_pw) / (b.max_pw - b.min_pw) END AS norm_pw,
    CASE WHEN b.max_eff = b.min_eff THEN 0.5 ELSE (b.max_eff - m.efficiency_wh_per_km) / (b.max_eff - b.min_eff) END AS norm_eff,
    CASE WHEN b.max_bat = b.min_bat THEN 0.5 ELSE (m.battery_capacity_kWh - b.min_bat) / (b.max_bat - b.min_bat) END AS norm_bat
  FROM metrics m CROSS JOIN bounds b
),

-- Compute a weighted composite score from the normalized metrics
-- Weights: .40 power-to-weight, .40 efficiency, .20 battery capacity
scored AS (
  SELECT
    car_model, brand,
    (0.4*norm_pw + 0.4*norm_eff + 0.2*norm_bat) AS composite_score
  FROM norms
)

-- Final output: rounded composite score, sorted descending, top 10 models
SELECT car_model, brand, ROUND(composite_score, 4) AS composite_score
FROM scored
ORDER BY composite_score DESC
LIMIT 10;
"""

# Execute the SQL and load the top-10 composite-ranked models into a pandas DataFrame
top10_composite = pd.read_sql(composite_query, conn)
print("\nTop 10 models by composite score:")
print(top10_composite)


Top 10 models by composite score:
                   car_model          brand  composite_score
0  Taycan Plus Sport Turismo        Porsche           0.5350
1                Taycan Plus        Porsche           0.5350
2                   CLA 250+  Mercedes-Benz           0.5317
3         Spring Electric 45          Dacia           0.4952
4                    #5 Pro+          Smart           0.4939
5                 #5 Premium          Smart           0.4939
6     IONIQ 9 Long Range RWD        Hyundai           0.4791
7          7X Long Range RWD          Zeekr           0.4768
8           EV9 99.8 kWh RWD            Kia           0.4565
9         EV6 Long Range 2WD            Kia           0.4444
